<a href="https://colab.research.google.com/github/Tung-Git-Hub/c3-4/blob/main/c3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.metrics import accuracy_score,classification_report
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

In [ ]:
#ham split train test
def shuffle_split_data(X, y):
    arr_rand = np.random.rand(X.shape[0])
    split = arr_rand < np.percentile(arr_rand, 70)
    #split = np.random.rand(X.shape[0]) < 0.7

    X_Train = X[split]
    y_Train = y[split]
    X_Test =  X[~split]
    y_Test = y[~split]
    return X_Train, y_Train, X_Test, y_Test

In [ ]:
df = pd.read_csv('/smoke_detection_iot.csv')
df.head()

,Unnamed: 0,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire Alarm
0,0,1654733331,20.000,57.36,0,400,12306,18520,939.735,0.0,0.0,0.0,0.0,0.0,0,0
1,1,1654733332,20.015,56.67,0,400,12345,18651,939.744,0.0,0.0,0.0,0.0,0.0,1,0
2,2,1654733333,20.029,55.96,0,400,12374,18764,939.738,0.0,0.0,0.0,0.0,0.0,2,0
3,3,1654733334,20.044,55.28,0,400,12390,18849,939.736,0.0,0.0,0.0,0.0,0.0,3,0
4,4,1654733335,20.059,54.69,0,400,12403,18921,939.744,0.0,0.0,0.0,0.0,0.0,4,0


In [ ]:
df.isnull().sum()

Unnamed: 0        0
UTC               0
Temperature[C]    0
Humidity[%]       0
TVOC[ppb]         0
eCO2[ppm]         0
Raw H2            0
Raw Ethanol       0
Pressure[hPa]     0
PM1.0             0
PM2.5             0
NC0.5             0
NC1.0             0
NC2.5             0
CNT               0
Fire Alarm        0
dtype: int64

In [ ]:
df.drop(columns = ['Unnamed: 0','UTC'],axis = 1,inplace = True)
df.drop(columns = ['NC1.0','PM1.0'],axis = 1,inplace =True)

In [ ]:
X=df.drop(columns = ['Fire Alarm'])
y=df['Fire Alarm']

In [ ]:
scale = MinMaxScaler()
X_scaled=pd.DataFrame(scale.fit_transform(X),columns=X.columns)

In [ ]:
#split data
x_train,y_train,x_test,y_test = shuffle_split_data(X_scaled,y)
print(x_train,y_train,x_test,y_test)

       Temperature[C]  Humidity[%]  TVOC[ppb]  eCO2[ppm]    Raw H2  \
3            0.513229     0.690971   0.000000        0.0  0.549282   
4            0.513412     0.681818   0.000000        0.0  0.553429   
5            0.513583     0.672975   0.000000        0.0  0.558533   
8            0.514120     0.652653   0.000000        0.0  0.567783   
9            0.514303     0.647223   0.000000        0.0  0.569378   
...               ...          ...        ...        ...       ...   
62622        0.485843     0.078343   0.010617        0.0  0.971930   
62623        0.488431     0.078188   0.010833        0.0  0.968102   
62624        0.491030     0.078653   0.010500        0.0  0.973206   
62625        0.493629     0.078343   0.010417        0.0  0.974482   
62626        0.496253     0.079584   0.010200        0.0  0.977033   

       Raw Ethanol  Pressure[hPa]     PM2.5     NC0.5         NC2.5       CNT  
3         0.579682       0.986125  0.000000  0.000000  0.000000e+00  0.000120  

In [ ]:
#train model và accuary tham khảo
smote = SMOTE()
x_train_smote, y_train_smote = smote.fit_resample(x_train, y_train)
model=RandomForestClassifier()
#fit model
model.fit(x_train_smote, y_train_smote)
y_pred_test=model.predict(x_test)
y_pred_train=model.predict(x_train_smote)
test_acc = accuracy_score(y_test,y_pred_test)
train_acc = accuracy_score(y_train_smote,y_pred_train)
print('test_acc: ', test_acc)
print('train_acc: ', train_acc)
print(pd.crosstab(y_test,y_pred_test))
print(classification_report(y_test, y_pred_test))

test_acc:  0.9998403321092129
train_acc:  1.0
col_0          0      1
Fire Alarm             
0           5315      0
1              3  13471
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5315
           1       1.00      1.00      1.00     13474

    accuracy                           1.00     18789
   macro avg       1.00      1.00      1.00     18789
weighted avg       1.00      1.00      1.00     18789

